# 1번. CNN & RNN

## 1.1 CNN cifar10

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os

c:\Users\minki\miniconda3\envs\dl_venv\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# seed 고정
def seed_everything(seed):
    #random.seed(seed)
    #os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
seed_everything(123)

In [3]:
# prepare Dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

batch_size = 256
valid_data_size = 10000
trainset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=True,
                                        download=True, transform=transform)
train, validset = torch.utils.data.random_split(trainset, [len(trainset) - valid_data_size, valid_data_size])
testset = torchvision.datasets.CIFAR10(root='./cifar10_data', train=False,
                                       download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(len(trainloader.dataset),len(validloader.dataset),len(testloader.dataset))

Files already downloaded and verified
Files already downloaded and verified
50000 10000 10000


trainig code

In [4]:
def training_model(model_name,model,train_loader,valid_loader,device):

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                            lr_lambda=lambda epoch: 0.97 ** epoch)
    n_epochs = 100

    train_loss_list , valid_loss_list = [] , []
    best_score = float('inf')

    model.train()
    patience = 0
    for epoch in range(n_epochs):
        train_loss = 0.0
        for data, target in train_loader:
            data , target = data.to(device) , target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
        scheduler.step()
        train_loss_list.append(train_loss/len(train_loader.dataset))

        # valid_data
        with torch.no_grad():
            valid_loss = 0.0
            for data, target in valid_loader:
                data , target = data.to(device) , target.to(device)
                output = model(data)
                loss = criterion(output, target)
                valid_loss += loss.item()*data.size(0)
            valid_loss_list.append(valid_loss/len(valid_loader.dataset))
        
            print('Epoch: {} \tTraining Loss: {:.6f} \t valid Loss: {:.6f}'.format(epoch+1, train_loss_list[-1],valid_loss_list[-1]))
            
        if valid_loss_list[-1] < best_score:
            best_score = valid_loss_list[-1]
            torch.save(model.state_dict(), os.path.join('./model_data/', '{}_{}.pth'.format(model_name,'best')))
            patience = 0
        else:
            patience += 1
            if patience == 8:
                return train_loss_list , valid_loss_list

def test_model(model_name,model,test_loader,device):

    model.load_state_dict(torch.load(os.path.join('./model_data/', '{}_{}.pth'.format(model_name,'best'))))

    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    criterion = nn.CrossEntropyLoss()

    model.eval()

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)
        _, pred = torch.max(output, 1)
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        for i in range(len(target.data)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        
            
    test_loss = test_loss/len(test_loader.dataset)
    print(f'{model_name}=========================================================')
    print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                str(i), 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (str(i)))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    print('==========================================================================')
    print()

AlexNet/VGG/ResNet/DenseNet 불러오기

In [2]:
'''
    [common preprocessing]
        resie 256*256 -> central_crop 224*224  
        mean=[0.485, 0.456, 0.406] std=[0.229, 0.224, 0.225]

    [AlexNet config]
        num_params : 61,100,840

    [VGG19]
        BN은 사용x 후에 Batch Normalization에서 적용
        num_params : 143,667,240

    [ResNet50]
        50 vs 101 vs 152가 성능이 비슷비슷해서 효율이 좋은 ResNet50 사용
        num_params : 25,557,032
    
    [DenseNet161]
        121부터 201까지 있는데 가장 성능이 좋은 161사용 
        특이하게도 layer수가 많아질수록 imageNet 성능이 안좋음
        num_params : 28,681,000
'''

from torchvision.models import alexnet, AlexNet_Weights 
from torchvision.models import vgg19, VGG19_Weights
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models import densenet161, DenseNet161_Weights

### Model finetuing

기존의 pretrained 모델은 224*224 image입력을 받음

cifar10은 32*32 image임

따라서 max pooling 모두 제거 , adaptivepooling으로 최종 feature size맞추기

ResNet과 DenseNet은 첫번째 pooling만 제거

In [4]:
model = densenet161(weights=DenseNet161_Weights)
model

c:\Users\minki\miniconda3\envs\dl_venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [6]:
def load_finetuned_model(model_name,update_conv=False):
    if model_name == 'alex' : 
        class New_alexnet(nn.Module):
            def __init__(self):
                super(New_alexnet, self).__init__()
            
                self.features = alexnet(weights=AlexNet_Weights).features
                new_features = [layer for layer in self.features if not isinstance(layer, torch.nn.MaxPool2d)]
                self.features = torch.nn.Sequential(*new_features)

                self.avgpool = alexnet(weights=AlexNet_Weights).avgpool


                self.classifier = alexnet(weights=AlexNet_Weights).classifier
                self.classifier[6] = nn.Linear(4096, 10)

                if not update_conv:
                    for param in self.features.parameters():
                        param.requires_grad = False

            def forward(self, x):
                x = self.features(x)
                x = self.avgpool(x)
                x = torch.flatten(x, 1)
                x = self.classifier(x)
                return x
        model = New_alexnet()
        return model



    elif model_name == 'vgg' : 
        class New_vgg19(nn.Module):
            def __init__(self):
                super(New_vgg19, self).__init__()
            
                self.features = vgg19(weights=VGG19_Weights).features
                new_features = [layer for layer in self.features if not isinstance(layer, torch.nn.MaxPool2d)]
                self.features = torch.nn.Sequential(*new_features)

                self.avgpool = vgg19(weights=VGG19_Weights).avgpool

                self.classifier = vgg19(weights=VGG19_Weights).classifier
                self.classifier[6] = nn.Linear(4096, 10)

                if not update_conv:
                    for param in self.features.parameters():
                        param.requires_grad = False

            def forward(self, x):
                x = self.features(x)
                x = self.avgpool(x)
                x = torch.flatten(x, 1)
                x = self.classifier(x)
                return x
        model = New_vgg19()
        return model
        

    elif model_name == 'resnet':
        model = resnet50(weights=ResNet50_Weights)
        if not update_conv:
            for param in model.parameters():
                param.requires_grad = False
        model.maxpool = nn.Identity()
        model.fc = nn.Linear(model.fc.in_features, 10)
        return model
    
    elif model_name == 'densenet':
        model = densenet161(weights=DenseNet161_Weights)
        if not update_conv:
            for param in model.parameters():
                param.requires_grad = False
        model.features.pool0 = nn.Identity()
        #del model.features.transition1[3]
        #del model.features.transition2[3]
        #del model.features.transition3[3]
    
        model.classifier = nn.Linear(model.classifier.in_features, 10)
        return model

In [7]:
# train

acc_dic = dict()
for model_name in ['alex','vgg','resnet','densenet']:
    training_dic = {
        'train_loss' : [] , 'valid_loss' : []
    }

    model = load_finetuned_model(model_name,update_conv=False)
    model.to(device)
    print(f'{model_name} training start')
    training_dic['train_loss'] , training_dic['valid_loss'] = training_model(model_name,model,trainloader,validloader,device)
    print(f'{model_name} training end')
    del model

c:\Users\minki\miniconda3\envs\dl_venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


alex training start
Epoch: 1 	Training Loss: 1.145617 	 valid Loss: 0.959292
Epoch: 2 	Training Loss: 0.917835 	 valid Loss: 0.914449
Epoch: 3 	Training Loss: 0.827853 	 valid Loss: 0.799198
Epoch: 4 	Training Loss: 0.762573 	 valid Loss: 0.694948
Epoch: 5 	Training Loss: 0.707732 	 valid Loss: 0.712366
Epoch: 6 	Training Loss: 0.662267 	 valid Loss: 0.631578
Epoch: 7 	Training Loss: 0.612045 	 valid Loss: 0.605816
Epoch: 8 	Training Loss: 0.571567 	 valid Loss: 0.549021
Epoch: 9 	Training Loss: 0.533611 	 valid Loss: 0.505672
Epoch: 10 	Training Loss: 0.504400 	 valid Loss: 0.491592
Epoch: 11 	Training Loss: 0.464479 	 valid Loss: 0.443604
Epoch: 12 	Training Loss: 0.435982 	 valid Loss: 0.432084
Epoch: 13 	Training Loss: 0.411360 	 valid Loss: 0.391992
Epoch: 14 	Training Loss: 0.382229 	 valid Loss: 0.357718
Epoch: 15 	Training Loss: 0.360526 	 valid Loss: 0.362410
Epoch: 16 	Training Loss: 0.338175 	 valid Loss: 0.345248
Epoch: 17 	Training Loss: 0.313486 	 valid Loss: 0.298507
Epo

c:\Users\minki\miniconda3\envs\dl_venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


vgg training start
Epoch: 1 	Training Loss: 1.145401 	 valid Loss: 0.970190
Epoch: 2 	Training Loss: 0.749656 	 valid Loss: 0.734735
Epoch: 3 	Training Loss: 0.602140 	 valid Loss: 0.572750
Epoch: 4 	Training Loss: 0.479379 	 valid Loss: 0.539440
Epoch: 5 	Training Loss: 0.385383 	 valid Loss: 0.300648
Epoch: 6 	Training Loss: 0.290644 	 valid Loss: 0.313017
Epoch: 7 	Training Loss: 0.209559 	 valid Loss: 0.209546
Epoch: 8 	Training Loss: 0.140207 	 valid Loss: 0.126249
Epoch: 9 	Training Loss: 0.097055 	 valid Loss: 0.076553
Epoch: 10 	Training Loss: 0.062212 	 valid Loss: 0.055169
Epoch: 11 	Training Loss: 0.042785 	 valid Loss: 0.042623
Epoch: 12 	Training Loss: 0.032881 	 valid Loss: 0.023752
Epoch: 13 	Training Loss: 0.025718 	 valid Loss: 0.022623
Epoch: 14 	Training Loss: 0.020422 	 valid Loss: 0.016919
Epoch: 15 	Training Loss: 0.017664 	 valid Loss: 0.021452
Epoch: 16 	Training Loss: 0.013805 	 valid Loss: 0.015601
Epoch: 17 	Training Loss: 0.013066 	 valid Loss: 0.013860
Epoc

c:\Users\minki\miniconda3\envs\dl_venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


resnet training start
Epoch: 1 	Training Loss: 3.002755 	 valid Loss: 2.676890
Epoch: 2 	Training Loss: 1.961365 	 valid Loss: 1.985894
Epoch: 3 	Training Loss: 1.764967 	 valid Loss: 1.509495
Epoch: 4 	Training Loss: 1.617515 	 valid Loss: 1.402955
Epoch: 5 	Training Loss: 1.561471 	 valid Loss: 1.784751
Epoch: 6 	Training Loss: 1.455560 	 valid Loss: 1.549729
Epoch: 7 	Training Loss: 1.389772 	 valid Loss: 2.413439
Epoch: 8 	Training Loss: 1.361637 	 valid Loss: 1.372078
Epoch: 9 	Training Loss: 1.288719 	 valid Loss: 1.601582
Epoch: 10 	Training Loss: 1.252534 	 valid Loss: 1.120561
Epoch: 11 	Training Loss: 1.211611 	 valid Loss: 1.697971
Epoch: 12 	Training Loss: 1.191753 	 valid Loss: 1.268425
Epoch: 13 	Training Loss: 1.138674 	 valid Loss: 1.093187
Epoch: 14 	Training Loss: 1.140171 	 valid Loss: 1.236370
Epoch: 15 	Training Loss: 1.094961 	 valid Loss: 1.089924
Epoch: 16 	Training Loss: 1.073446 	 valid Loss: 1.219008
Epoch: 17 	Training Loss: 1.056819 	 valid Loss: 1.106549
E

c:\Users\minki\miniconda3\envs\dl_venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


densenet training start
Epoch: 1 	Training Loss: 2.012583 	 valid Loss: 1.663062
Epoch: 2 	Training Loss: 1.466452 	 valid Loss: 1.567900
Epoch: 3 	Training Loss: 1.368739 	 valid Loss: 1.329501
Epoch: 4 	Training Loss: 1.286107 	 valid Loss: 1.609982
Epoch: 5 	Training Loss: 1.275280 	 valid Loss: 1.217026
Epoch: 6 	Training Loss: 1.225415 	 valid Loss: 1.428956
Epoch: 7 	Training Loss: 1.210452 	 valid Loss: 1.481516
Epoch: 8 	Training Loss: 1.193926 	 valid Loss: 1.349013
Epoch: 9 	Training Loss: 1.161560 	 valid Loss: 1.236330
Epoch: 10 	Training Loss: 1.134310 	 valid Loss: 1.456598
Epoch: 11 	Training Loss: 1.128933 	 valid Loss: 1.160844
Epoch: 12 	Training Loss: 1.129263 	 valid Loss: 1.125249
Epoch: 13 	Training Loss: 1.102995 	 valid Loss: 1.090898
Epoch: 14 	Training Loss: 1.105141 	 valid Loss: 1.117067
Epoch: 15 	Training Loss: 1.085370 	 valid Loss: 1.117603
Epoch: 16 	Training Loss: 1.083056 	 valid Loss: 1.151514
Epoch: 17 	Training Loss: 1.065254 	 valid Loss: 1.185742

In [8]:
# test
for model_name in ['alex','vgg','resnet','densenet']:
    model = load_finetuned_model(model_name)
    model.to(device)
    test_model(model_name,model,testloader,device)

alex=========================================================
Test Loss: 0.920206

Test Accuracy of     0: 78% (786/1000)
Test Accuracy of     1: 82% (827/1000)
Test Accuracy of     2: 68% (686/1000)
Test Accuracy of     3: 55% (559/1000)
Test Accuracy of     4: 73% (735/1000)
Test Accuracy of     5: 68% (688/1000)
Test Accuracy of     6: 82% (822/1000)
Test Accuracy of     7: 80% (807/1000)
Test Accuracy of     8: 85% (854/1000)
Test Accuracy of     9: 83% (830/1000)

Test Accuracy (Overall): 75% (7594/10000)

vgg=========================================================
Test Loss: 0.972574

Test Accuracy of     0: 85% (851/1000)
Test Accuracy of     1: 91% (915/1000)
Test Accuracy of     2: 71% (712/1000)
Test Accuracy of     3: 66% (661/1000)
Test Accuracy of     4: 76% (764/1000)
Test Accuracy of     5: 68% (682/1000)
Test Accuracy of     6: 83% (837/1000)
Test Accuracy of     7: 82% (820/1000)
Test Accuracy of     8: 89% (894/1000)
Test Accuracy of     9: 89% (895/1000)

Test Accur

||test_acc|error_rate|train_loss|valid_loss|epoch|
|------|---|---|---|---|---|
|AlexNet|%|%|0.0603|0.0530|88|
|AlexNet with conv train|%|%||||
|VGG19|%|%|0.0022|0.0023|53|
|VGG19 with conv train|%|%||||
|ResNet50|%|%|1.0441|1.0274|18|
|ResNet50 with conv train|%|%||||
|DenseNet161|%|%||||
|DenseNet161 with conv train|%|%||||

In [97]:
0.97**50*0.1

0.02180653753474073